In [1]:
import utils.datareader as datareader
import utils.visualizer as visualizer
from models.transformer import *

In [2]:
datafolder = 'C://Users//marku//Desktop//krafthack_files'

targets = [f'Bolt_{n+1}_Tensile' for n in range(6)]
learning_rate = 3e-3 # optimizer
learning_rate_annealing = 0.1 # how much the learning rate is annealed every episode
episodes = 5000 # how often it is iterated over the entire dataset
inputlength = 100
batchsize = 50


Read and scale datasets:

In [3]:
df, dfT = datareader.df_from_folder(datafolder)
df, dfT, scaler = datareader.normalize_input(df, dfT)

features = list(dfT.columns) # find features

Remove overhead data, find the nan timestamps and mask nans:

In [4]:
# remove overhead:
df = df[targets+features] 
# find nan timestamps:
nans = np.array(df[targets].isna().product(1)) * np.arange(df.shape[0]) 
nans = nans[nans != 0]
# mask nans:
df = df.fillna(0)
dfT = dfT.fillna(0)

Set up model:

In [5]:
net = Network(df.shape[1], len(targets), maxlen=inputlength)
net.train() # starting in training mode
optim = torch.optim.Adam(net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [8]:
plot_episode = 100 # which episodes to plot
losshistory = []
for e in range(episodes):
    # zero gradients out:
    net.zero_grad() 
    # sample a batch:
    x, y = sample(df, nans, targets, inputlength, batchsize, device=net.device)
    # predict:
    y_pred = net(x)      
    # calculate losses:
    loss = loss_fn(y_pred, y)
    # backpropagation:
    loss.backward()
    optim.step()       
    # save losses:
    losshistory.append(loss.detach().cpu().numpy())  
    if e % plot_episode == 0:
        print(f'episode {e+1}: {losshistory[-1]}')


episode 1: 4.664834976196289


KeyboardInterrupt: 